In [1]:
import ogr

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

In [4]:
rdd = sc.textFile("./data/dltb.tsv")

### 直接过滤掉字段头和不能构建成要素的行（清洗脏数据）

In [5]:
def filterHeader(line):
    geoline = line.split("\t")
    try:
        return ogr.CreateGeometryFromWkt(geoline[1]).IsValid()
    except:
        return False

### 清洗之前，包括头，一共193行

In [6]:
rdd.count()

193

### 过滤掉之后剩下192行了

In [7]:
rdd2 = rdd.filter(lambda line : filterHeader(line))

In [8]:
rdd2.count()

192

## 构建一个由几何要素和需要的属性组成的元祖

In [9]:
def myBuilderGeom(line):
    geoline = line.split("\t")
    geo = ogr.CreateGeometryFromWkt(geoline[1])
    return (geo,geoline[3])

In [10]:
rdd3 = rdd2.map(lambda line : myBuilderGeom(line))

### 查看前面五行

In [33]:
rdd3.take(5)

[(<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001EFBB845F60> >,
  '旱地'),
 (<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001EFBB845F90> >,
  '园地'),
 (<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001EFBB845E70> >,
  '林地'),
 (<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001EFBB8DC450> >,
  '园地'),
 (<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x000001EFBB8DC060> >,
  '园地')]

### 进行面积统计

In [32]:
rdd3.map(lambda feat: (feat[1],
                       feat[0].GetArea())).reduceByKey(lambda x,y:x+y).collect()

[('旱地', 1062114.5405737045),
 ('园地', 974189.4862199697),
 ('农村居民点用地', 39557.91353517026),
 ('其他独立建设用地', 257737.88656154094),
 ('林地', 4250805.430884263)]